In [2]:
import pandas as pd

df = pd.read_csv('../data/sample_products.csv')

df.head()

,product_id,name,category,brand,price,tags,description
0,1,PageTurner Electronics #1,Electronics,PageTurner,439.23,battery|wireless|bluetooth|4K,PageTurner electronics item designed for batte...
1,2,PageTurner Beauty #2,Beauty,PageTurner,292.59,anti-aging|SPF|vitamin-c|sensitive|skincare,PageTurner beauty item designed for anti-aging...
2,3,PageTurner Fashion #3,Fashion,PageTurner,373.81,summer|formal|casual|leather,PageTurner fashion item designed for summer an...
3,4,Silk&Co Home #4,Home,Silk&Co,402.76,eco-friendly|durable|minimal,Silk&Co home item designed for eco-friendly an...
4,5,PageTurner Beauty #5,Beauty,PageTurner,337.18,SPF|hydrating|anti-aging|skincare|vitamin-c,PageTurner beauty item designed for SPF and hy...


In [3]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['category'] = encoder.fit_transform(df['category'])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming 'product_name' is a text feature
vectorizer = TfidfVectorizer(max_features=100)
X = vectorizer.fit_transform(df['name'])


In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer

scaler = StandardScaler()

df['price_scaled'] = scaler.fit_transform(df[['price']])

df['category_price_interaction'] = df['category'] * df['price_scaled']


description_vectorizer = TfidfVectorizer(max_features=100)
description_features = description_vectorizer.fit_transform(df['description'])

mlb = MultiLabelBinarizer()
tags_features = mlb.fit_transform(df['tags'])

In [ ]:
# 02_feature_engineering.ipynb

# Step 1: Import Necessary Libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Step 2: Load Data
# Load the product and interaction data
product_data = pd.read_csv('../data/sample_products.csv')
interaction_data = pd.read_csv('../data/raw/sample_interactions.csv')

# Step 3: Exploratory Data Analysis (EDA)
# Check for missing values in product data
print("Missing values in product data:")
print(product_data.isnull().sum())

# Check for missing values in interaction data
print("\nMissing values in interaction data:")
print(interaction_data.isnull().sum())

# Step 4: Text Feature Extraction (TF-IDF)
# We will extract features from the 'tags' column in the product data using TF-IDF Vectorizer
tfidf = TfidfVectorizer(max_features=100)
product_tags_tfidf = tfidf.fit_transform(product_data['tags'])

# Check the shape of the TF-IDF matrix
print(f"\nTF-IDF feature shape: {product_tags_tfidf.shape}")

# Step 5: Feature Transformation (Scaling)
# Normalize price feature to ensure it's on the same scale
scaler = StandardScaler()
product_data['price_scaled'] = scaler.fit_transform(product_data[['price']])

# Step 6: Combine Text and Numerical Features
# We will combine the TF-IDF features with the scaled price feature to create the final feature set
final_features = np.hstack([product_tags_tfidf.toarray(), product_data[['price_scaled']].values])

# Step 7: Prepare Training Data
# Create the 'interaction' target variable from event_type
interaction_map = {
    'view': 0,          # Neutral interaction
    'like': 1,          # Positive interaction
    'add_to_cart': 2    # Strong interaction
}

# Apply the mapping to create the 'interaction' column
interaction_data['interaction'] = interaction_data['event_type'].map(interaction_map)

# Merge interaction and product data based on 'product_id'
train_data = pd.merge(interaction_data, product_data, on='product_id')

# Check the first few rows of the merged dataset
print("\nSample of the merged training data:")
print(train_data.head())

# Step 8: Select Features (X) and Target (y)
X = train_data[['user_id', 'product_id']]  # Example features: user_id and product_id
y = train_data['interaction']  # Now the 'interaction' column

# Step 9: Split Data into Training and Test Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 10: Display Preprocessed Data
print("\nSample of the processed training data:")
print(train_data.head())

# Optionally, save the processed features and labels
np.save('processed_features.npy', final_features)
np.save('labels.npy', y.values)

# The code ends here for feature engineering. You can continue to the next steps in your notebook to train the recommendation model.


Missing values in product data:
product_id     0
name           0
category       0
brand          0
price          0
tags           0
description    0
dtype: int64

Missing values in interaction data:
event_id      0
user_id       0
product_id    0
event_type    0
timestamp     0
dtype: int64

TF-IDF feature shape: (60, 47)

Sample of the merged training data:
   event_id  user_id  product_id   event_type            timestamp  \
0       316        2          14         view  2025-10-01T08:44:00   
1       252       16          46  add_to_cart  2025-10-01T11:19:00   
2       235        1          57         like  2025-10-01T12:25:00   
3         7        4          23  add_to_cart  2025-10-01T15:02:00   
4        22        1           1         view  2025-10-01T15:49:00   

   interaction                       name     category       brand   price  \
0            0         NovaTech Books #14        Books    NovaTech  187.62   
1            2        HomeNest Beauty #46       Beauty    Ho